# Lab 2
## Introduction
This lab introduces slope fields and a numerical DE solver, an improved version of
Euler’s Method. Using these techniques involves a number of commands.

### Slope fields
Plot the slope field for the differential equation
\begin{align*}
\frac{\mathrm{d}y}{\mathrm{d}x} = x - y
\end{align*}
for $-1<x<5$ and $-2<y<4$.

Start by importing the NumPy and Plotly methods we will need. This week, we will start to make more sophisticated plots using Plotly graph objects. We will also need a SciPy method. SciPy extends the capabilities of NumPy.

In [ ]:
from plotly.figure_factory import create_quiver
from plotly import graph_objs as go
from numpy import meshgrid, linspace, sqrt
from numpy.testing import assert_almost_equal

from scipy.integrate import odeint

Now plot the slope field. A slope field is a special type of _quiver_ plot. We create NumPy arrays that say where to plot the line segments (`x` and `y`) and arrays to point them in the right direction (`1/L` and `S/L`). The `layout.update` method is optional, but makes the plot prettier and easier to understand.

In [ ]:
x, y = meshgrid(linspace(-1, 5, 25), linspace(-2, 4, 25))
S = x - y
L = sqrt(1 + S**2)
fig = create_quiver(x, y, 1/L, S/L, scale=0.3, arrow_scale=1e-16)
fig.layout.update(title_text="Slopefield for dy/dx = x - y",
                  yaxis=dict(scaleanchor='x', scaleratio=1))
fig.show('png')

`1/L` and `S/L` in the `create_quiver` command set the $x$ and $y$ lengths (components) of the line segment at each point in the grid.

Note that NumPy operates element-wise by default, so `x - y` creates an array of differences, and `S/L` creates an array of quotients. For `1/L`, NumPy does something special called _broadcasting_. It assumes that you meant "divide an array of ones by the elements of `L`".

The slope of the line segment is then $(S/L)/(1/L) = S$, and the length is
\begin{align*}
\sqrt{\left(\frac{1}{L}\right)^2 + \left(\frac{S}{L}\right)^2} &= \sqrt{\frac{1+S^2}{L^2}}\\
&= 1.
\end{align*}

### Numerical/graphical solution of an initial-value problem
Plot the (approximate) solution to the initial-value problem
\begin{align*}
\frac{\mathrm{d}y}{\mathrm{d}x} = x - y\qquad y(-1)=0
\end{align*}
for $-1 < x <5$. Find $y(5)$.

Here we use a numerical DE solver `scipy.integrate.odeint` (which we imported as `odeint`), an improved version of Euler’s Method. To use `odeint`, we need to define the differential equation in a _function_ and then feed it to `odeint`.

First define the function. Note that in Python, [white space is important](https://xkcd.com/353/). That is, you have to indent the contents of your function or Python will complain. Most of the time Jupyter will figure out your intentions and auto-indent.

In [ ]:
def diff_eq(y, x):
    return x - y

- The `def` keyword tells Python you would like to define a function.
- In this case the function is called `diff_eq` and takes arguments `y` and `x`.
- The `return` statement tells Python what you would like to return.
- When you stop indenting, the function is over.

Note that `odeint` expects the function (`diff_eq` here) to take (at least) two arguments, where the first (`y` here) is the dependent variable and the second (`x` here) is the independent variable, even if these variables do not appear in the RHS of the DE.

Now ask `odeint` to generate a solution to our DE.

In [ ]:
x = linspace(-1, 5, 61)
y = odeint(diff_eq, 0, x).flatten()

- `linspace` creates an array of (`61`, in this case) equally-spaced elements.
- `odeint` calculates `y` for each value of `x`.
- In Python, functions are objects like any other. In this case we pass `diff_eq` as an argument to `odeint`.
- The second argument to `odeint` (`0` here) is the initial value of $y$. It must correspond to the first value of `x`.
- `odeint` returns a 2D array. We need a 1D array for plotting so we `flatten` it.

The following will plot `x` and `y` in a line diagram. Notice the syntax is more complicated than last week. We create the figure, add the line to it, then show it. The advantage of this syntax is that it can be extended to show more than one line on a graph, as we will see below.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.show('png')

Finally, to calculate $y(5)$, we realise that the values calculated by `odeint` are stored in the array `y`. So display `y`.

In [ ]:
y

Here we just want the last value. We can grab the last element of the array with `y[-1]`. (`y[-2]` gives the second last element.)

In [ ]:
y[-1]

`x[-1]` is th elast element of `x`. Check it too.

In [ ]:
x[-1]

Now we will plot multiple (approximate) solutions on the same graph. We do that by just adding each line as we go. Does the resulting figure make sense?

In [ ]:
fig = go.Figure()
x = linspace(-1, 5, 61)
y = odeint(diff_eq, 0, x).flatten()
fig.add_trace(go.Scatter(x=x, y=y))
y = odeint(diff_eq, 2, x).flatten()
fig.add_trace(go.Scatter(x=x, y=y))
y = odeint(diff_eq, -2, x).flatten()
fig.add_trace(go.Scatter(x=x, y=y))
fig.layout.update(showlegend=False)
fig.show('png')

Now let’s put the slope field and the numerical solutions together. Copy and paste the code from above where we created the quiver plot into the cell below. Delete the `fig.show()` command and replace it with the code from the above cell. Delete the command where we create an empty figure (`fig = go.Figure()`), because we want to add our line plots to the slope field.

If you have done it properly, the result should look like this:

![](images/week-2.png)

## Exercises

Start by importing the tools we will need.

### Slope field and DE solution plot

Plot on the one figure the slopefield for the DE
\begin{align*}
\frac{\mathrm{d} y}{\mathrm{d} x} = 2.5y (1 − y)\qquad y(0) = 0.5,
\end{align*}
and the solutions $y(x)$ with $y(0) = 0.2$, $y(0) = 0.5$ and $y(0) = 0.8$.

Start by writing down a new definition for `diff_eq` below. Do not change the function's name or inputs.

In [ ]:
def diff_eq(y, x):
    ### diff_eq implementation goes here

If you have implemented `diff_eq` correctly, the following should print "nice job".

In [ ]:
assert_almost_equal(diff_eq(0.4, 0), 0.6)
assert_almost_equal(diff_eq(0.4, 10), 0.6)
assert_almost_equal(diff_eq(1, 0), 0)
print("nice job")

Now create your graph. Note that you will have to redefine `S` (from the lab). You can do that using your new definition for `diff_eq` or by writing out the RHS of the equation again.

You will also have to change your definition of the meshgrid for the slopefield and the domain and initial values in the `odeint` commands. You want about 21 steps in the x and y ranges in meshgrid. If you change the scaling factor from 0.3 to 0.04 in `create_quiver`, you will get a better slope field.

Create the plot for the region $0 < x < 1$ and $0 < y < 1$.

### Solution at a point
What is $y(1)$ if $y(0)=0.8$?